# Mock QCNN Training

In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

2023-02-23 18:14:47.108727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Mock 2 (50iter,10batch)/Model2.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Mock 2 (50iter,10batch)/Model3.pt'


# Mock QCNN

Hyperparameters

In [3]:
steps = 50
learning_rate = 0.1
batch_size = 10
ansatz = 'TTN'

QCNN

In [4]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [5]:
def statepreparation(x, Trained):
    if Trained == False:
        embedding.Noisy_Four_QuantumEmbedding1(x)
    elif Trained == 'Model2':
        model2.load_state_dict(torch.load(PATH_Model2))
        x = model2(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x)
    elif Trained == 'Model3':
        model3.load_state_dict(torch.load(PATH_Model3))
        x = model3(torch.tensor(x))
        embedding.Noisy_Four_QuantumEmbedding2(x[0])
    

@qml.qnode(dev)
def Noisy_QCNN_classifier(params, x, Trained):
    param1 = params[0:2]
    param2 = params[2:4]

    statepreparation(x, Trained)
    embedding.U_TTN(param1, wires=[0,1])
    embedding.U_TTN(param1, wires=[3,2])
    embedding.U_TTN(param2, wires=[1,2])
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, Trained):
    preds = [Noisy_QCNN_classifier(weights, x, Trained) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, Trained):

    weights = np.random.random(4, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, Trained),
                                                     weights)
        loss_history.append(cost_new)
        print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

# Model 2

In [6]:

feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

2023-02-23 18:14:54.065496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_22124/461388107.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646991213/work/torch/csrc/utils/tensor_new.cpp:233.)
  X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)


In [7]:

Loss_histories_not_trained, weights_not_trained, Loss_histories_Model2, weights_Model2 = [], [], [], []

for i in range(3):
    loss_not_trained, weight_not_trained = circuit_training(X_train, Y_train, False)
    loss_Model2, weight_Model2 = circuit_training(X_train, Y_train, 'Model2')

    Loss_histories_not_trained.append(loss_not_trained)
    Loss_histories_Model2.append(loss_Model2)

    weights_not_trained.append(weight_not_trained)
    weights_Model2.append(weight_Model2)

Loss_histories_not_traiend, Loss_histories_Model2 =  np.array(Loss_histories_not_trained), np.array(Loss_histories_Model2)


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_not_trained.txt', 'a')
f.write(str(Loss_histories_not_trained))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_not_trained.txt', 'a')
f.write(str(weights_not_trained))
f.close()


f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model2.txt', 'a')
f.write(str(Loss_histories_Model2))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model2.txt', 'a')
f.write(str(weights_Model2))
f.close()

/usr/local/anaconda3/envs/QC/lib/python3.10/site-packages/pennylane/math/utils.py:223: UserWarning: Contains tensors of types {'autograd', 'torch'}; dispatch will prioritize TensorFlow, PyTorch, and  Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(
/usr/local/anaconda3/envs/QC/lib/python3.10/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


iteration:  0  cost:  0.471484375
iteration:  1  cost:  0.4322265625
iteration:  2  cost:  0.4798828125
iteration:  3  cost:  0.466796875
iteration:  4  cost:  0.53740234375
iteration:  5  cost:  0.51650390625
iteration:  6  cost:  0.46962890625
iteration:  7  cost:  0.50078125
iteration:  8  cost:  0.4703125
iteration:  9  cost:  0.50078125
iteration:  10  cost:  0.4724609375
iteration:  11  cost:  0.5
iteration:  12  cost:  0.4634765625
iteration:  13  cost:  0.44775390625
iteration:  14  cost:  0.49765625
iteration:  15  cost:  0.47373046875
iteration:  16  cost:  0.512109375
iteration:  17  cost:  0.505078125
iteration:  18  cost:  0.46416015625
iteration:  19  cost:  0.4720703125
iteration:  20  cost:  0.5099609375
iteration:  21  cost:  0.47626953125
iteration:  22  cost:  0.47998046875
iteration:  23  cost:  0.49697265625
iteration:  24  cost:  0.49140625
iteration:  25  cost:  0.50322265625
iteration:  26  cost:  0.494140625
iteration:  27  cost:  0.4892578125
iteration:  28  c

/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_22124/3053866874.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model2(torch.tensor(x))


iteration:  0  cost:  0.48671875
iteration:  1  cost:  0.48505859375
iteration:  2  cost:  0.48125
iteration:  3  cost:  0.458203125
iteration:  4  cost:  0.449609375
iteration:  5  cost:  0.4486328125
iteration:  6  cost:  0.44677734375
iteration:  7  cost:  0.4484375
iteration:  8  cost:  0.4318359375
iteration:  9  cost:  0.41923828125
iteration:  10  cost:  0.4322265625
iteration:  11  cost:  0.423828125
iteration:  12  cost:  0.4298828125
iteration:  13  cost:  0.39541015625
iteration:  14  cost:  0.40361328125
iteration:  15  cost:  0.3896484375
iteration:  16  cost:  0.3720703125
iteration:  17  cost:  0.36376953125
iteration:  18  cost:  0.39404296875
iteration:  19  cost:  0.34794921875
iteration:  20  cost:  0.34677734375
iteration:  21  cost:  0.3390625
iteration:  22  cost:  0.33642578125
iteration:  23  cost:  0.3203125
iteration:  24  cost:  0.31025390625
iteration:  25  cost:  0.30185546875
iteration:  26  cost:  0.31865234375
iteration:  27  cost:  0.30869140625
iterati

# Model3

## 0. Getting Started

In [8]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

In [9]:
X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [10]:
Loss_histories_Model3, weights_Model3 = [], []
for i in range(3):

    loss_Model3, weight_Model3 = circuit_training(X_train, Y_train, 'Model3')

    Loss_histories_Model3.append(loss_Model3)
    weights_Model3.append(weight_Model3)

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/Loss_Model3.txt', 'a')
f.write(str(Loss_histories_Model3))
f.close()

f = open('/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Mock/weights_Model3.txt', 'a')
f.write(str(weights_Model3))
f.close()

/var/folders/g3/xz_ycw3d0qdd8s4r2rdd6m940000gn/T/ipykernel_22124/3053866874.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = model3(torch.tensor(x))


iteration:  0  cost:  0.62666015625
iteration:  1  cost:  0.63662109375
iteration:  2  cost:  0.59697265625
iteration:  3  cost:  0.59091796875
iteration:  4  cost:  0.59765625
iteration:  5  cost:  0.56494140625
iteration:  6  cost:  0.54521484375
iteration:  7  cost:  0.5357421875
iteration:  8  cost:  0.50556640625
iteration:  9  cost:  0.5080078125
iteration:  10  cost:  0.49423828125
iteration:  11  cost:  0.5087890625
iteration:  12  cost:  0.49501953125
iteration:  13  cost:  0.50546875
iteration:  14  cost:  0.49052734375
iteration:  15  cost:  0.483984375
iteration:  16  cost:  0.48544921875
iteration:  17  cost:  0.49755859375
iteration:  18  cost:  0.491796875
iteration:  19  cost:  0.4888671875
iteration:  20  cost:  0.4845703125
iteration:  21  cost:  0.481640625
iteration:  22  cost:  0.45771484375
iteration:  23  cost:  0.455078125
iteration:  24  cost:  0.43134765625
iteration:  25  cost:  0.4697265625
iteration:  26  cost:  0.451171875
iteration:  27  cost:  0.46259765